In [8]:
from deap import base, creator, tools, algorithms
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.lines as lines
import random
from mpl_toolkits.basemap import Basemap

print "PHY905 Project 3: Traveling Salesman"
print "by Steve Hughey"
print "5/4/15"

# Read cities file
cities = [line.strip() for line in open('Cities.txt')]
Ncities = len(cities)
LatLon = [line.strip().strip('{}') for line in open('LatLon.txt')]
city_coords = np.zeros((Ncities,2))
print "Number of cities: %d" % Ncities

for i in range(Ncities):
    tmp = LatLon[i].split(',')
    city_coords[i,0] = np.float(tmp[0])
    city_coords[i,1] = np.float(tmp[1])
    print "[%-2d] %-30s\t(%f,%f)" % (i,cities[i],city_coords[i,0],city_coords[i,1])

# Read distance matrix and store in distMat
# All distances in distMat in units of km
distMat = np.zeros((Ncities,Ncities))
# pLinks: all possible links between cities
pLinks = np.zeros((Ncities*(Ncities-1)/2,2))
dst = [line.strip().split('\t') for line in open('distMatrix.mat')]
cnt = 0
for i in range(Ncities):
    for j in range(i+1,Ncities):
        distMat[i,j] = np.float(dst[i][j-i-1])/1000.0
        distMat[j,i] = np.float(dst[i][j-i-1])/1000.0
        pLinks[cnt,0] = i
        pLinks[cnt,1] = j
        cnt += 1
print pLinks

plt.figure()
plt.imshow(np.log10(distMat))
plt.colorbar()
plt.show()

PHY905 Project 3: Traveling Salesman
by Steve Hughey
5/4/15
Number of cities: 40
[0 ] New York City, New York       	(40.664274,-73.938500)
[1 ] Los Angeles, California       	(34.019394,-118.410825)
[2 ] Chicago, Illinois             	(41.837551,-87.681844)
[3 ] Houston, Texas                	(29.780472,-95.386342)
[4 ] Philadelphia, Pennsylvania    	(40.009375,-75.133346)
[5 ] Phoenix, Arizona              	(33.572162,-112.087966)
[6 ] San Antonio, Texas            	(29.472403,-98.525142)
[7 ] San Diego, California         	(32.815300,-117.134993)
[8 ] Dallas, Texas                 	(32.794176,-96.765503)
[9 ] San Jose, California          	(37.296867,-121.819306)
[10] Indianapolis, Indiana         	(39.777995,-86.145838)
[11] Jacksonville, Florida         	(30.337019,-81.661302)
[12] San Francisco, California     	(37.759881,-122.437392)
[13] Austin, Texas                 	(30.307182,-97.755996)
[14] Columbus, Ohio                	(39.984799,-82.985044)
[15] Fort Worth, Texas       

/usr/lib/python2.7/site-packages/IPython/kernel/__main__.py:42: RuntimeWarning: divide by zero encountered in log10


In [4]:
# Get the distance of a complete circuit around the cities
# Goal is to minimize this function over all possible paths
def pathDistance(path):
    dist = 0.0
    for i in range(Ncities-1):
        dist += distMat[path[i],path[i+1]]
    dist += distMat[path[-1],path[0]]
    return dist

# Create the toolbox for the algorithm
print "Creating DEAP structures..."
toolbox = base.Toolbox()
creator.create("FitnessMin",base.Fitness,weights=(-1.0,))
creator.create("Individual",list,fitness=creator.FitnessMin)

# the register method creates an alias for some function with arguments
# and stores it in the DEAP toolbox
toolbox.register("indices",np.random.permutation,Ncities)
toolbox.register("individual",tools.initIterate,creator.Individual,toolbox.indices)
toolbox.register("population",tools.initRepeat,list,toolbox.individual)
toolbox.register("mate",tools.cxOrdered)
toolbox.register("mutate",tools.mutShuffleIndexes,indpb=0.05)

# Define the fitness function (distance) using the toolbox.individual
# object, output must be iterable tuple
def fitnessFunction(individual):
    return (pathDistance(individual),)

toolbox.register("evaluate",fitnessFunction)
toolbox.register("select",tools.selTournament,tournsize=4)
#toolbox.register("select",tools.selBest)
#toolbox.register("select",tools.selRoulette)
print "Done"

# Generate an initial population
popSize = 200
pop = toolbox.population(n=popSize)
result, log = algorithms.eaSimple(pop,toolbox,cxpb=0.8,mutpb=0.15,ngen=1000,verbose=True)
best_individual = tools.selBest(result, k=1)[0]
print "Fitness of the best individual: ", fitnessFunction(best_individual)

Creating DEAP structures...
Done
gen	nevals
0  	200   
1  	176   
2  	165   
3  	162   
4  	170   
5  	167   
6  	164   
7  	152   
8  	163   
9  	162   
10 	169   
11 	160   
12 	174   
13 	160   
14 	152   
15 	157   
16 	167   
17 	183   
18 	169   
19 	172   
20 	165   
21 	168   
22 	155   
23 	174   
24 	165   
25 	159   
26 	169   
27 	156   
28 	153   
29 	152   
30 	177   
31 	156   
32 	177   
33 	152   
34 	170   
35 	165   
36 	174   
37 	180   
38 	172   
39 	160   
40 	161   
41 	170   
42 	167   
43 	163   
44 	156   
45 	165   
46 	168   
47 	173   
48 	176   
49 	174   
50 	173   
51 	170   
52 	171   
53 	158   
54 	162   
55 	158   
56 	176   
57 	165   
58 	161   
59 	167   
60 	178   
61 	161   
62 	159   
63 	152   
64 	174   
65 	162   
66 	170   
67 	167   
68 	154   
69 	172   
70 	178   
71 	159   
72 	165   
73 	168   
74 	151   
75 	168   
76 	168   
77 	162   
78 	168   
79 	177   
80 	153   
81 	165   
82 	161   
83 	168   
84 	167   
85 	173   
86 	162   

/usr/lib/python2.7/site-packages/deap/creator.py:132: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
/usr/lib/python2.7/site-packages/deap/creator.py:132: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


In [5]:
# Draw the map
lats = [city_coords[best_individual[i],0] for i in range(Ncities)]
lons = [city_coords[best_individual[i],1] for i in range(Ncities)]
lats.append(city_coords[best_individual[0],0])
lons.append(city_coords[best_individual[0],1])
m = Basemap(width=5500000,height=3000000,projection='lcc',
            resolution='l',lat_1=45.,lat_2=45.,lat_0=39,lon_0=-95.)
m.readshapefile('in101503','in101503')
m.drawcoastlines()
m.drawcountries()
m.drawstates()
m.drawmapboundary(fill_color='aqua')
m.fillcontinents(color='gray',lake_color='aqua')
x, y = m(lons,lats)
m.plot(x,y, '-o', markersize=10, linewidth=1, color='k', markerfacecolor='g')
bbox_props = dict(boxstyle="round",fc="w",ec="0.5",alpha=0.5)
for i in range(Ncities):
    icity = best_individual[i]
    a = plt.text(x[i],y[i],cities[icity], ha='center',\
      va='center',bbox=bbox_props,fontsize=8)
    plt.arrow(x[i],y[i],x[i+1]-x[i],y[i+1]-y[i],width=1,head_width=50000,color='b')
plt.show()